In [1]:
!pip install datasets scikit-learn matplotlib numpy -q >> None

import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay

In [2]:
!git clone https://github.com/Rishit-dagli/Astroformer.git

Cloning into 'Astroformer'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (580/580), done.
remote: Compressing objects: 100% (355/355), done.
remote: Total 580 (delta 222), reused 570 (delta 218), pack-reused 0 (from 0)
Receiving objects: 100% (580/580), 2.39 MiB | 3.68 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [23]:
from datasets import load_dataset, Image
import os
from PIL import Image as PILImage

# Load the dataset
dataset_name = "matthieulel/galaxy10_decals"
galaxy_dataset = load_dataset(dataset_name)

# Make sure the image column is properly cast as Image type
if not isinstance(galaxy_dataset["train"].features["image"], Image):
    galaxy_dataset = galaxy_dataset.cast_column("image", Image())

# Create directories for each class
class_names = {
    0: "Disturbed_Galaxies",
    1: "Merging_Galaxies",
    2: "Round_Smooth_Galaxies",
    3: "In-between_Round_Smooth_Galaxies",
    4: "Cigar_Shaped_Smooth_Galaxies",
    5: "Barred_Spiral_Galaxies",
    6: "Unbarred_Tight_Spiral_Galaxies",
    7: "Unbarred_Loose_Spiral_Galaxies",
    8: "Edge-on_Galaxies_without_Bulge",
    9: "Edge-on_Galaxies_with_Bulge"
}

output_dir = "galaxy10_images"
os.makedirs(output_dir, exist_ok=True)
for class_idx, class_name in class_names.items():
    os.makedirs(os.path.join(output_dir, f"{class_idx}_{class_name}"), exist_ok=True)

# Extract, resize, and save images
for i, example in enumerate(galaxy_dataset["train"]):
    img = example["image"]
    label = example["label"]
    class_name = class_names[label]
    
    # Resize the image to 224x224 pixels using ANTIALIAS for better quality
    img_resized = img.resize((224, 224), PILImage.LANCZOS)
    
    # Save the resized image
    img_path = os.path.join(output_dir, f"{label}_{class_name}", f"galaxy_{i}.png")
    img_resized.save(img_path)
    
    # Print progress
    if i % 1000 == 0:
        print(f"Processed {i} images")

print(f"Extracted and resized {len(galaxy_dataset['train'])} images to 224x224 pixels in {output_dir}")


Processed 0 images
Processed 1000 images
Processed 2000 images
Processed 3000 images
Processed 4000 images
Processed 5000 images
Processed 6000 images
Processed 7000 images
Processed 8000 images
Processed 9000 images
Processed 10000 images
Processed 11000 images
Processed 12000 images
Processed 13000 images
Processed 14000 images
Processed 15000 images
Extracted and resized 15962 images to 224x224 pixels in galaxy10_images


In [5]:
%cd Astroformer/pytorch-image-models/

/app/svf_knowledge_editing/Astroformer/pytorch-image-models


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
!pip install --upgrade torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 36.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 10.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 MB 10.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 9.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 6.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━

In [28]:
!sh distributed_train.sh 1 galaxy10_images --train-split 0.8 --val-split 0.2 --model astroformer_4 --num-classes 10 --img-size 224 --in-chans 3 --input-size 3 256 256 --batch-size 10 --grad-accum-steps 1 --opt adamw --sched cosine --lr-base 2e-5 --lr-cycle-decay 1e-2 --lr-k-decay 1 --warmup-lr 1e-5 --epochs 300 --warmup-epochs 5 --mixup 0.8 --smoothing 0.1 --drop 0.1 --save-images --output result_ours/astroformer_4_galaxy10

Training with a single process on 1 device (cuda:0).
/usr/local/lib/python3.10/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model astroformer_4 created, param count:241083274
Data processing configuration for current model + dataset:
	input_size: (3, 256, 256)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225)
	crop_pct: 0.875
	crop_mode: center
Learning rate (3.952847075210474e-06) calculated from base learning rate (2e-05) and effective global batch size (10) with sqrt scaling.
AMP not enabled. Training in float32.
Scheduled epochs: 300. LR stepped per epoch.
Traceback (most recent call last):
  File "/app/svf_knowledge_editing/Astroformer/pytorch-image-models/train.py", line 1075, in <module>
    main()
  File 

In [11]:
!df -h /dev/shm

Filesystem      Size  Used Avail Use% Mounted on
shm              64M  4.0K   64M   1% /dev/shm
